<a href="https://colab.research.google.com/github/jaekyoungkim/floor_DL/blob/main/%EB%B0%91%EB%B0%94%EB%8B%A5%EB%B6%80%ED%84%B0%EC%8B%9C%EC%9E%91%ED%95%98%EB%8A%94_%EB%94%A5%EB%9F%AC%EB%8B%9D_(ch5_6).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/youbeebee/deeplearning_from_scratch

Cloning into 'deeplearning_from_scratch'...
remote: Enumerating objects: 405, done.
remote: Total 405 (delta 0), reused 0 (delta 0), pack-reused 405
Receiving objects: 100% (405/405), 58.06 MiB | 41.84 MiB/s, done.
Resolving deltas: 100% (74/74), done.


# ch 5. 오차역전파법 

In [ ]:
# 앞장에서 신경망의 가중치 매개변수의 기울기는 수치미분을 사용해 계산함
# 수치 미분은 단순하고 구현하기도 쉽지만 계산시간이 오래걸린다는게 단점 
# 가중치 매개변수의 기울기를 효율적으로 계산하는 오차역전파법
# 계산 그래프를 사용하는 이유 : 전체가 아무리 복잡해도 각 노드에서는 단순한 계산에 집주앟여 문제를 단순화 시킬 수 있음 + 중간결과 저장가능 + 역전파 통해 미분을 효율적으로 계산
# 연쇄법칙 

In [ ]:
# 사과쇼핑 예를 파이썬으로 구현하기
# 모든 게층은 forward() backward() 라는 공통의 메서드를 갖도록 구현함
# forward 순전파, backward 역전파
# 곱셈 계층은 mullayer
class MuLayer:
  def __init__(self): # x,y,를 초기화
    self.x = None
    self.y = None
  
  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y
    return out
  
  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x
    return dx, dy

In [ ]:
apple = 100
apple_num =2 
tax = 1.1

# 계층들
mul_apple_layer = MuLayer()
mul_tax_layer = MuLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [ ]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [ ]:
# 덧셈계층
class AddLayer:
  def __init__(self):  # 덧셈 계층에서는 초기화가 필요없음 아무일도 안함
    pass
  
  def forward(self, x, y):
    out = x+y
    return out 
  
  def backward(self, dout): 
    dx = dout * 1 
    dy = dout * 1
    return dx, dy

In [ ]:
# 사과 2개, 귤3개구입에 대한 역전파
apple = 100
apple_num =2 
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MuLayer()
mul_orange_layer = MuLayer()
add_apple_orange_layer =AddLayer()
mul_tax_layer = MuLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price , dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price =add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [ ]:
# 활성화 함수 계층 구성하기
# 1. relu
# x > 0 : 역전파는 상류의 값을 그대로 하류로 흘림
# x < 0 : 역전파는 하류로 신호를 보내지 않음
class Relu:
  def __init__(self):
    self.mask = None  #  true, false로 구성된 넘파이 배열 
  
  def forward(self, x):
    self.mask = (x<=0) # x 0이하는 true, 0 초과는 FALSE
    out = x.copy()
    out[self.mask] = 0 
    return out
  
  def backward(self, dout):
    dout[self.mask] = 0 
    dx = dout
    return dx 

In [ ]:
import numpy as np
x = np.array([[1.0, 0.5], [-2.0, 3.0]])
print(x)

mask = (x <= 0)
print(mask) 

[[ 1.   0.5]
 [-2.   3. ]]
[[False False]
 [ True False]]


In [ ]:
# 2. sigmoid 계층
# x-> y  : y^2 * exp(-x) 가 하류 노드로 전파됨  / y(1-y):순전파의 출력(y)만으로 계산할수도있음
class Sigmoid:
  def __init__(self):
    self.out = None
  
  def forward(self, x):
    out = 1/(1+np.exp(-x))
    self.out = out 
    return out
  
  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out
    return dx



In [ ]:
# Affine / softmax계층 구현하기
# 1. affine 계층
# 신경망의 순전파때 수행하는 행렬의 곱은 기하학에서는 어파인 변환이라고 함



In [ ]:
# 배치용 affine 계층
# 입력데이터로 x하나만을 고려하지 않고, 데이터 n개를 묶어 순전파하는경우(묶은 데이터를 배치라고 함)
x_dot_W = np.array([[0,0,0], [10,10,10]])
B = np.array([1,2,3])

x_dot_W 

array([[ 0,  0,  0],
       [10, 10, 10]])

In [ ]:
x_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [ ]:
dY = np.array([[1,2,3], [4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [ ]:
dB = np.sum(dY, axis=0); dB # 같은 위치에 있는 원소끼리 합쳐줌, 0 번째 축으로 합치기
# 편향의 역전파는 n=2라고 가정하면, 그 두 데이터에 대한 미분을 데이터마다 더해수 구함

array([5, 7, 9])

In [ ]:
np.sum(dY, axis=1) # 1번째 축으로 합치기

array([ 6, 15])

In [ ]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)  # 오버플로 대책
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y


def cross_entropy_error(y, t):
    delta = 1e-7  # 0일때 -무한대가 되지 않기 위해 작은 값을 더함
    return -np.sum(t * np.log(y + delta))


In [ ]:
class Affine:
  def __init__(self, W, b):
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None

  def forward(self,x):
    self.x = x
    out = np.dot(x, self.W) + self.b
    return out

  def backward(self, dout):
    dx = np.dot(dout, self.W.T)  # x에대한 역전파
    self.dW = np.dot(self.x.T, dout) #w에대한 역전파
    self.db = np.sum(dout, axis= 0) # 배치결과들을 합치기
    return dx

In [ ]:
# 2. softmax-with-loss 계층
# 소프트맥스 함수는 입력값을 정규화하여 출력함
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None
    self.y = None
    self.t = None
  
  def forward(self, x, t):
    self.t = t
    self.y = softmax(t)
    self.loss = cross_entropy_error(self.y , self.t)
    return self.loss
  
  def backward(self, dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size # 전파하는 값을 배치의 수로 나눠서 데이터 1개당 오차를 앞 계층으로 전파함
    return dx


In [ ]:
# 오차역전파 구현하기
# 수치미분은 구현하기는 쉽지만 계산이 오래 걸림 -> 오차역전파법을 이용하면 느린 수치미분과 달리 기울기를 효율적으로 빠르게 구할 수 있음
# numerical_gradient : 수치미분방식
# gradient : 오차역전파법

In [ ]:
import sys , os
#sys.path.append('/content/deeplearning_from_scratch/common')
sys.path.append('/content/deeplearning_from_scratch/')
import numpy as np 
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [ ]:
class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std= 0.01):
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size , hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)

    #계층생성 
    self.layers = OrderedDict() # 순서가 있는 딕셔너리 
    # 순전파때는 추가한 순서대로 각 계층에 forward메서드를 호출하기만 하면 됨
    # 역전파때는 계층을 반대 순서로 호출하면됨
    self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
    self.lastlayer = SoftmaxWithLoss()

  def predict(self,x): #예측
    for layer in self.layers.values():
      x = layer.forward(x)
    
    return x

  # x 입력데이터, t 정답 레이블
  def loss(self, x, t):
    y= self.predict(x)
    return self.lastLayer.forward(y,t)
  
  def accuracy(self, x, t):
    y=self.predict(x)
    y= np.argmax(y, axis=1)
    if t.ndim !=1 : t= np.argmax(t, axis=1)

    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy


  def numerical_gradient(self, x, t):
    loss_W = lambda W : self.loss(x,t)
    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

    return grads
    


  def gradient(self, x, t):
    # 순전파
    self.loss(x,t)

    # 역전파
    dout = 1
    dout = self.lastLayer.backward(dout)
    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)

    # 결과저장
    grads = {}
    grads['W1'] = self.layers['Affine1'].dW
    grads['b1'] = self.layers['Affine1'].db
    grads['W2'] = self.layers['Affine2'].dW
    grads['b2'] = self.layers['Affine2'].db 
    
    return grads   

In [ ]:
# 오차역전파법으로 구한 기울기 검증하기
# 1. 수치미분 
# 2. 해석적으로 수식을 풀어 구하는 방법, 매개변수가 많아도 효율적으로 계산할 수 있음
# 수치미분은 오차역전파법을 정확히 구현했는지 확인하기 위해 필요함
# 수치미분의 장점은 구현하기 쉽다는 것
# 수치미분구현에는 버그가 숨어있기 어려운반면, 오차역전파법은 구현하기 복잡해서 종종 실수를 하곤 함
# 기울기확인 : 1,2방법의 결과가 같은지 확인

In [ ]:
from dataset.mnist import load_mnist
from common.two_layer_net import TwoLayerNet  # ch5 에 들어있는 two_layer_net 파일 common으로 옮기기

In [ ]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)
network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size= 10)
x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 차이의 절댓값을 구한 후 그 절댓값들의 평균을 구함
for key in grad_numerical.keys():
  diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]))
  print(key + ":" + str(diff))  # 차이가 매우작음을 알 수 있음

W1:2.2260574784664928e-13
b1:6.023729865606287e-13
W2:7.829423153370412e-13
b2:1.2034817170603062e-10


In [ ]:
# 오차역전파를 사용한 학습 구현하기
network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size=10)
iters_num = 10000
train_size = x_train.shape[0] ;train_size # 60000


60000

In [ ]:
batch_size = 100
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size / batch_size, 1)
for i in range(iters_num):
  batch_mask = np.random.choice(train_size, batch_size) 
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  #오차역전파법으로 기울기 구하기
  grad = network.gradient(x_batch, t_batch)

  # 갱신
  for key in ('W1','b1','W2','b2'):
    network.params[key] -= learning_rate * grad[key]
  
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)

  if i% iter_per_epoch ==0:
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print(train_acc, test_acc)

0.14916666666666667 0.1508
0.9021666666666667 0.9051
0.9216333333333333 0.9233
0.9330333333333334 0.9354
0.94485 0.9425
0.9507166666666667 0.9488
0.9563833333333334 0.9514
0.9596333333333333 0.9545
0.9636 0.9579
0.9663333333333334 0.9608
0.9667166666666667 0.9609
0.9706833333333333 0.965
0.9728 0.9657
0.9729666666666666 0.9665
0.9751166666666666 0.9683
0.97425 0.9672
0.97835 0.9711


# ch 6. 학습 관련 기술들 

In [ ]:
# 최적화 : 신경망의 최적화는 굉장히 어려운 문제 , 매개변수 공간은 매우넓고 복잡해서 최적의 솔루션은 쉽게 못찾음
# 매개변수 갱신
# SGD보다 좋은 최적화 방법들을 알아보고자 함


In [3]:
# 1. SGD
class SGD:
  def __init__(self, lr = 0.01):
    self.lr = lr
  
  def update(self, params, grads):
    for key in params.keys():
      params[key] -= self.lr * grads[key]

In [4]:
#network = TwoLayerNet(...)
#optimizer = SGD() #  이부분을 바꾸면됨
#for i in range(10000):
# x_batch, t_batch = get_mini_batch(...)
# grads = network.gradient(x+batch, t_batch)
# params = network.params
# optimizer.update(params, grads) 

# SGD의 단점 : 단순하고 구현도 쉽지만, 비등방성함수에서는 탐색경로가 비효율적 ,무작정 기울어진 방향으로 진행하는 단순한 방식

In [5]:
# 2. Momentum (운동량)
class Momentum():
  def __init__(self, lr = 0.01, momentum = 0.9): # v 물체의 속도
    self.lr = lr
    self.momentum = momentum
    self.v = None
  
  def update(self, params, grads):
    if self.v is None:
      self.v = {}
      for key, val in params.items():
        self.v[key] = np.zeros_like(val)  # 초기 array생성해줌
      
    for key in params.keys():
      self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
      params[key] += self.v[key]  # params = Weight


In [ ]:
# 3. AdaGrad
# 신경망 학습에서 학습률값이 중요
# 이 값이 너무 작으면 학습시간이 너무 길어지고, 너무크면 발산하여 학습이 제대로 이뤄지지 않음
# 학습률을 정하는 효과적기술로 학습률 감소가 있음
# 학습을 진행하면서 학습률을 점차 줄여가는 방법
# 처음에는 크게 학습하다가 조금씩 작게 학습한다는 것 (실제 신경망 학습에 자주 사용됨)
# 학습률을 서서히 낮추는 가장 간단한 방법은 매개변수 전체의 학습률 값을 일괄적으로 낮추는것
# 각각의 매개변수에 맞춤형값을 만들어줌
# 학습을 진행할수록 갱신강도가 약해짐 실제로 무한히 계속 학습하다보면  어느순간 갱신량이 0이되어 전혀 갱신되지 않음
class AdaGrad:
  